$\require{mhchem}$
Torkel's model is similar to Jiri's model, except that:
 - It only captures energy sterss with $RsbP$, and doesn't capture enviromental stress with $RspU$. 
 - It captures individual production and degradation of each species.
 
## Model

Coding preabmle below

In [29]:
import multiprocessing
from tqdm.notebook import tqdm
import numpy as np
import scipy.stats as st
import numba

# Plotting modules
import bokeh.io
import bokeh.plotting
import biocircuits

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
%load_ext line_profiler

def sample_discrete(probabilities):
    # Generate random number
    r_num = np.random.rand()
    
    # Find assosiated propensity
    ind = 0
    prob_sum = 0.0
    while prob_sum < r_num:
        prob_sum += probabilities[ind]
        ind += 1
    return ind - 1

def gillespie_draw(propensity_func, propensities, variables, t, args=()):
    # 1) Calculate new propensites
    propensity_func(propensities, variables, t, *args)
    
    # 2) Sum propensities
    propensity_sum = propensities.sum()
#     print(propensities)
#     print(propensity_sum)
    # 3) Compute next time
    dt = np.random.exponential(1.0 / propensity_sum)
    
    # 4) Compute reaction probabilites from propensities
    reaction_probabilites = propensities / propensity_sum
    
    # 5) Draw a random reaction
    chosen_reaction = sample_discrete(reaction_probabilites)
    
    return chosen_reaction, dt

# Now we will construct a functions to run the gilespie algorithim
def gillespie_algorithim(propensity_function, update_array, initial_conditions, time_points, args=()):
    # Initialise output
    pop_out = np.empty((len(time_points), update_array.shape[1]), dtype = int)

    # Initialise and perform simulation
    index = 0
    time_index = 1
    time = time_points[0]
    variables = initial_conditions.copy()
    pop_out[0,:] = variables
    propensities = np.zeros(update_array.shape[0])
    while index < len(time_points):
        while time < time_points[time_index]:
            # Draw reaction and time step
            reaction, dt = gillespie_draw(propensity_function, propensities, variables, time, args)

            # Update variables
            variables_previous = variables.copy()
            variables += update_array[reaction, :]

            # Increment time
            time += dt

        # Update index
        index = np.searchsorted(time_points > time, True)

        # Update the variables
        pop_out[time_index:min(index, len(time_points))] = variables_previous

        # Increment index
        time_index = index

    return pop_out

Loading BokehJS ...

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Production:
$$\require{mhchem}$$

| Reaction                        | Rate                                                       | Propensity                                                 |
| :-----------------------------: | :--------------------------------------------:             | :--------------------------------------------:             |
|$$\ce{\emptyset -> \sigma^{B}}$$ | $$v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$             | $$v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$             |
|$$\ce{\emptyset -> W}$$          | $$\lambda_{W} v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$ | $$\lambda_{W} v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$ |
|$$\ce{\emptyset -> V}$$          | $$\lambda_{V} v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$ | $$\lambda_{V} v_0 \frac{1 + F[\sigma^B]}{K + [\sigma^B]}$$ |

Dimerisation:

| Reaction                        | Rate                                                       | Propensity                                                 |
| :-----------------------------: | :--------------------------------------------:             | :--------------------------------------------:             |
|$$\ce{2W -> W_{2}}$$             | $$k_{Bw}$$                                                 | $$k_{Bw} \frac{[W]^2}{2!}$$                                |


Binding:

| Reaction                                     | Rate                                          | Propensity                                                 |
| :-----------------------------:              | :--------------------------------------------:| :--------------------------------------------:             |
|$$\ce{W_{2} + V -> W_{2}V}$$                  | $$k_{B1}$$                                    | $$k_{B1} [W_2][V]$$                                        |
|$$\ce{W_{2}V + V -> W_{2}V_{2}}$$             | $$k_{B2}$$                                    | $$k_{B2} [W_2V][V]$$                                       |
|$$\ce{W_{2} + \sigma^{B} -> W_{2}\sigma^{B}}$$| $$k_{B3}$$                                    | $$k_{B3} [W_2][\sigma^{B}]$$                               |
|$$\ce{P + V^{p} -> PV^{p}}$$                  | $$k_{B5}$$                                    | $$k_{B5} [P][V^{p}]$$                                      |

Partner switching:

| Reaction                                          | Rate                                          | Propensity                                                 |
| :-----------------------------:                   | :--------------------------------------------:| :--------------------------------------------:             |
|$$\ce{W_{2}\sigma^{B} + V -> W_{2}V + \sigma^{B}}$$| $$k_{B4}$$                                    | $$k_{B4} [W_2\sigma^{B}][V]$$                              |
|$$\ce{W_{2}V + \sigma^{B} -> W_{2}\sigma^{B} + V}$$| $$k_{D4}$$                                    | $$k_{D4} [W_2V][\sigma^{B}]$$                              |

Dissociation:

| Reaction                                     | Rate                                          | Propensity                                                 |
| :-----------------------------:              | :--------------------------------------------:| :--------------------------------------------:             |
|$$\ce{W_{2} -> 2W}$$                          | $$k_{Dw}$$                                    | $$k_{Dw} [W_2]$$                                           |
|$$\ce{W_{2}V -> W_{2} + V}$$                  | $$k_{D1}$$                                    | $$k_{D1} [W_{2}V]$$                                        |
|$$\ce{W_{2}V_{2} -> W_{2}V + V}$$             | $$k_{D2}$$                                    | $$k_{D2} [W_{2}V_{2}]$$                                    |
|$$\ce{W_{2}\sigma^{B} -> W_{2} + \sigma^{B}}$$| $$k_{D3}$$                                    | $$k_{D3} [W_{2}\sigma^{B}]$$                               |
|$$\ce{PV^{p} -> P + V^{p}}$$                  | $$k_{D5}$$                                    | $$k_{D5} [PV^{p}]$$                                        |

Phosphorylation:

| Reaction                                     | Rate                                          | Propensity                                                 |
| :-----------------------------:              | :--------------------------------------------:| :--------------------------------------------:             |
|$$\ce{W_{2}V -> W_{2} + V^{p}}$$              | $$k_{K1}$$                                    | $$k_{K1} [W_{2}V]$$                                        |
|$$\ce{W_{2}V_{2} -> W_{2}V + V^{p}}$$         | $$k_{K2}$$                                    | $$k_{K2} [W_{2}V_{2}]$$                                    |

Dephosphorylation:

| Reaction                                     | Rate                                          | Propensity                                                 |
| :-----------------------------:              | :--------------------------------------------:| :--------------------------------------------:             |
|$$\ce{PV^{p} -> P + V}$$                      | $$k_{P}$$                                     | $$k_{P} [PV^{p}]$$                                         |

Degradation/Dilution:

| Reaction                             | Rate                                           | Propensity                                                  |
| :-----------------------------:      | :--------------------------------------------: | :--------------------------------------------:              |
|$$\ce{\sigma^{B} -> \emptyset}$$      | $$k_{deg}$$                                    | $$k_{deg} [\sigma^{B}]$$                                    |
|$$\ce{W -> \emptyset}$$               | $$k_{deg}$$                                    | $$k_{deg} [W]$$                                             |
|$$\ce{W_{2} -> \emptyset}$$           | $$k_{deg}$$                                    | $$k_{deg} [W_{2}]$$                                         |
|$$\ce{W_{2}\sigma^{B} -> \emptyset}$$ | $$k_{deg}$$                                    | $$k_{deg} [W_{2}\sigma^{B}]$$                               |
|$$\ce{W_{2}V -> \emptyset}$$          | $$k_{deg}$$                                    | $$k_{deg} [W_{2}V]$$                                        |
|$$\ce{W_{2}V_{2} -> \emptyset}$$      | $$k_{deg}$$                                    | $$k_{deg} [W_{2}V_{2}]$$                                    |
|$$\ce{V -> \emptyset}$$               | $$k_{deg}$$                                    | $$k_{deg} [V]$$                                             |
|$$\ce{V^{p} -> \emptyset}$$           | $$k_{deg}$$                                    | $$k_{deg} [V^{p}]$$                                         |
|$$\ce{PV^{p} -> P}$$                  | $$k_{deg}$$                                    | $$k_{deg} [PV^{p}]$$                                        |

In [51]:
#                                sig_B,        W,        V,        P,      W_2,     W_2V,   W_2V_2, W_2sig_B,      V_p,     PV_p
SigB_acc_update = np.array([[        1,        0,        0,        0,        0,        0,        0,        0,        0,        0],  # Production of sig_B                            
                            [        0,        1,        0,        0,        0,        0,        0,        0,        0,        0],  # Production of W                                 
                            [        0,        0,        1,        0,        0,        0,        0,        0,        0,        0],  # Production of V                                 
                            [        0,       -2,        0,        0,        1,        0,        0,        0,        0,        0],  # Dimerisation of 2W to W_2                                        
                            [        0,        0,       -1,        0,       -1,        1,        0,        0,        0,        0],  # Binding W_2 with V                                   
                            [        0,        0,       -1,        0,        0,       -1,        1,        0,        0,        0],  # Binding W_2V with V                               
                            [       -1,        0,        0,        0,       -1,        0,        0,        1,        0,        0],  # Binding W_2 with sig_B                              
                            [        0,        0,        0,       -1,        0,        0,        0,        0,       -1,        1],  # Binding P with V_p                                              
                            [        1,        0,       -1,        0,        0,        1,        0,       -1,        0,        0],  # Partner switching W_2sig_B + V to W_2V + sig_B                                                 
                            [       -1,        0,        1,        0,        0,       -1,        0,        1,        0,        0],  # Partner switching W_2V + sig_B to W_2sig_B + V                                                  
                            [        0,        2,        0,        0,       -1,        0,        0,        0,        0,        0],  # Dissociation of W_2 to 2W                                                  
                            [        0,        0,        1,        0,        1,       -1,        0,        0,        0,        0],  # Dissociation of W_2V to W_2 + V                                                  
                            [        0,        0,        1,        0,        0,        1,       -1,        0,        0,        0],  # Dissociation of W_2V_2 to W_2V + V                                                  
                            [        1,        0,        0,        0,        1,        0,        0,       -1,        0,        0],  # Dissociation of W_2sig_B to W_2 + sig_B                                                 
                            [        0,        0,        0,        1,        0,        0,        0,        0,        1,       -1],  # Dissociation of PV_p to P + V_p                    
                            [        0,        0,        0,        0,        1,       -1,        0,        0,        1,        0],  # Phosphorylation of W_2V to W_2 + V_p                                                  
                            [        0,        0,        0,        0,        0,        1,       -1,        0,        1,        0],  # Phosphorylation of W_2V_2 to W_2V + V_p                                                  
                            [        0,        0,        1,        1,        0,        0,        0,        0,        0,       -1],  # Dephosphorylation of PV_p to P + V                                         
                            [       -1,        0,        0,        0,        0,        0,        0,        0,        0,        0],  # Degradation/Dilution of sig_B                                                
                            [        0,       -1,        0,        0,        0,        0,        0,        0,        0,        0],  # Degradation/Dilution of W                                             
                            [        0,        0,        0,        0,       -1,        0,        0,        0,        0,        0],  # Degradation/Dilution of W_2                                           
                            [        0,        0,        0,        0,        0,        0,        0,       -1,        0,        0],  # Degradation/Dilution of W_2sig_B                                              
                            [        0,        0,        0,        0,        0,       -1,        0,        0,        0,        0],  # Degradation/Dilution of W_2V                                            
                            [        0,        0,        0,        0,        0,        0,       -1,        0,        0,        0],  # Degradation/Dilution of W_2V_2                                           
                            [        0,        0,       -1,        0,        0,        0,        0,        0,        0,        0],  # Degradation/Dilution of V               
                            [        0,        0,        0,        0,        0,        0,        0,        0,       -1,        0],  # Degradation/Dilution of V_p                                    
                            [        0,        0,        0,        1,        0,        0,        0,        0,        0,       -1]], # Degradation/Dilution of PV_p to P                                             
                      dtype = int)

The model then uses the following parameters:

| Parameter         | Value                        | Value (seconds)                          | Description                                                 | 
| :---------------: | :------------:               |  :-------------:                         | :-----------:                                               |
| $$v_0$$           | $$0.4 \mu M hr^{-1}$$        | $$1.44\times 10^{3} \mu M s^{-1}$$       | Operon base activity                                        |
| $$F$$             | $$30$$                       | $$30$$                                   | Operon fold change                                          |
| $$K$$             | $$0.2 \mu M$$                | $$0.2 \mu M$$                            | $\sigma^B$ binding affinity for operon                      |  
| $$\lambda_W$$     | $$4$$                        | $$4$$                                    | $W$ to $\sigma^B$ relative production rate                  |
| $$\lambda_V$$     | $$4.5$$                      | $$4.5$$                                  | $V$ to $\sigma^B$ relative production rate                  |
| $$k_{Bw}$$        | $$3600 \mu M^{-1} hr^{-1} $$ | $$1.296\times 10^{7} \mu M^{-1} s^{-1}$$ | $W$ dimerisation rate                                       |
| $$k_{Dw}$$        | $$18 hr^{-1} $$              | $$6.48\times 10^{4}  s^{-1}$$            | $W_2$ dissosiation rate                                     |
| $$k_{B1}$$        | $$3600 \mu M^{-1} hr^{-1} $$ | $$1.296\times 10^{7} \mu M^{-1} s^{-1}$$ | $W_2$ to $V$ binding rate                                   |
| $$k_{B2}$$        | $$3600 \mu M^{-1} hr^{-1} $$ | $$1.296\times 10^{7} \mu M^{-1} s^{-1}$$ | $W_2V$ to $V$ binding rate                                  |
| $$k_{B3}$$        | $$3600 \mu M^{-1} hr^{-1} $$ | $$1.296\times 10^{7} \mu M^{-1} s^{-1}$$ | $W_2$ to $\sigma^B$ binding rate                            |
| $$k_{B4}$$        | $$1800 \mu M^{-1} hr^{-1} $$ | $$6.48\times 10^{6}  \mu M^{-1} s^{-1}$$  | $W_2\sigma^B$ partner switching from $\sigma^B$ to $V$ rate|
| $$k_{B5}$$        | $$3600 \mu M^{-1} hr^{-1} $$ | $$1.296\times 10^{7} \mu M^{-1}s^{-1}$$  | $P$ to $V^{p}$ binding rate                                 |
| $$k_{D1}$$        | $$18 hr^{-1} $$              | $$6.48\times 10^{4}  s^{-1}$$            | $W_2V$ dissosation rate                                     |
| $$k_{D2}$$        | $$18 hr^{-1} $$              | $$6.48\times 10^{4}  s^{-1}$$            | $W_2V_2$ dissosation rate                                   |
| $$k_{D3}$$        | $$18 hr^{-1} $$              | $$6.48\times 10^{4}  s^{-1}$$            | $W_2\sigma^B$ dissosation rate                              |
| $$k_{D4}$$        | $$1800 \mu M^{-1} hr^{-1} $$ | $$6.48\times 10^{6} \mu M^{-1} s^{-1}$$  | $W_2V$ partner switching from $V$ to $\sigma^B$ rate        |
| $$k_{D5}$$        | $$18 hr^{-1} $$              | $$6.48\times 10^{4}  s^{-1}$$            | $PV^{p}$ dissosation rate                                   |
| $$k_{K1}$$        | $$36 hr^{-1} $$              | $$1.296\times 10^{5} s^{-1}$$            | $W_2V$ phosphorylation rate                                 |
| $$k_{K2}$$        | $$36 hr^{-1} $$              | $$1.296\times 10^{5} s^{-1}$$            | $W_2V_2$ phosphorylation rate                               |
| $$k_{P}$$         | $$180 hr^{-1} $$             | $$6.48\times 10^{5}  s^{-1}$$            | $PV^{p}$ dephosphorylation rate                             |
| $$k_{deg}$$       | $$0.7 hr^{-1} $$             | $$2.52\times 10^{3}  s^{-1}$$            | Degradation/dilution rate                                   |
| $$p_{init}$$      | $$0.007 \mu M $$             | $$0.007 \mu M $$                         | Phosphate concentration during stress                       |



In [52]:
# @numba.njit
def SigB_propensity(propensities, species, t, v_0, F, K, lambda_W, lambda_V, k_Bw, k_Dw, k_B1, k_B2, k_B3, k_B4, k_B5, k_D1, k_D2, k_D3, k_D4, k_D5, k_K1, k_K2, k_P, k_deg):
    sig_B, W, V, P, W_2, W_2V, W_2V_2, W_2sig_B, V_p, PV_p = species
    
    propensities[0]  = v_0 * ((1 + F*sig_B)/(K + sig_B)) # Production of sig_B                           
    propensities[1]  = lambda_W * v_0 * ((1 + F*sig_B)/(K + sig_B)) # Production of W                               
    propensities[2]  = lambda_V * v_0 * ((1 + F*sig_B)/(K + sig_B)) # Production of V                               
    propensities[3]  = k_Bw * W * (W-1) / 2 # Dimerisation of 2W to W_2                     
    propensities[4]  = k_B1 * W_2 * V # Binding W_2 with V                            
    propensities[5]  = k_B2 * W_2V * V # Binding W_2V with V                           
    propensities[6]  = k_B3 * W_2 * sig_B # Binding W_2 with sig_B                        
    propensities[7]  = k_B5 * P * V_p # Binding P with V_p                            
    propensities[8]  = k_B4 * W_2sig_B * V # Partner switching W_2sig_B + V to W_2V + sig_B
    propensities[9]  = k_D4 * W_2V * sig_B # Partner switching W_2V + sig_B to W_2sig_B + V
    propensities[10] = k_Dw * W_2 # Dissociation of W_2 to 2W                     
    propensities[11] = k_D1 * W_2V # Dissociation of W_2V to W_2 + V               
    propensities[12] = k_D2 * W_2V_2 # Dissociation of W_2V_2 to W_2V + V            
    propensities[13] = k_D3 * W_2sig_B # Dissociation of W_2sig_B to W_2 + sig_B       
    propensities[14] = k_D5 * PV_p # Dissociation of PV_p to P + V_p               
    propensities[15] = k_K1 * W_2V # Phosphorylation of W_2V to W_2 + V_p          
    propensities[16] = k_K2 * W_2V_2 # Phosphorylation of W_2V_2 to W_2V + V_p       
    propensities[17] = k_P * PV_p # Dephosphorylation of PV_p to P + V            
    propensities[18] = k_deg * sig_B # Degradation/Dilution of sig_B                 
    propensities[19] = k_deg * W # Degradation/Dilution of W                     
    propensities[20] = k_deg * W_2 # Degradation/Dilution of W_2                   
    propensities[21] = k_deg * W_2sig_B # Degradation/Dilution of W_2sig_B              
    propensities[22] = k_deg * W_2V # Degradation/Dilution of W_2V                  
    propensities[23] = k_deg * W_2V_2 # Degradation/Dilution of W_2V_2                
    propensities[24] = k_deg * V # Degradation/Dilution of V                     
    propensities[25] = k_deg * V_p # Degradation/Dilution of V_p                   
    propensities[26] = k_deg * PV_p # Degradation/Dilution of PV_p to P             

In [53]:
#            v_0,  F,      K, lambda_W, lambda_V,    k_Bw,   k_Dw,    k_B1,    k_B2,    k_B3, k_B4,    k_B5,   k_D1,   k_D2,   k_D3,   k_D4,   k_D5,    k_K1,    k_K2,    k_P,  k_deg
# args = ( 1.44e-3, 30, 0.2e-6,        4,      4.5, 1.296e1, 6.48e4, 1.296e1, 1.296e1, 1.296e1, 6.48, 1.296e1, 6.48e4, 6.48e4, 6.48e4,   6.48, 6.48e4, 1.296e5, 1.296e5, 6.48e5, 2.52e3)
# args = ( 1.44e-3, 30, 0.2e-6,        4,      4.5, 1.296e7, 6.48e4, 1.296e7, 1.296e7, 1.296e7, 6.48, 1.296e1, 6.48e4, 6.48e4, 6.48e4, 6.48e6, 6.48e4, 1.296e5, 1.296e5, 6.48e5, 2.52e3)
#From Torkel v_0,  F,   K, lambda_W, lambda_V, k_Bw, k_Dw, k_B1, k_B2, k_B3, k_B4, k_B5, k_D1, k_D2,  k_D3, k_D4, k_D5, k_K1, k_K2, k_P,  k_deg
args = (     0.4, 30, 0.2,        4,      4.5, 3600,   18, 3600, 3600, 3600, 1800, 3600,   18,   18,    18, 1800,   18,   36,   36, 180, 0.7)
# Low Stress                    sig_B, W, V,  P, W_2, W_2V, W_2V_2, W_2sig_B, V_p, PV_p
initial_conditions = np.array([     0, 0, 0,  0,   0,    0,      0,        0,   0,    0],  dtype = int)
# High Stress                    sig_B, W, V,        P, W_2, W_2V, W_2V_2, W_2sig_B, V_p, PV_p
#initial_conditions = np.array([     0, 0, 0, 0.80e-6,   0,    0,      0,        0,   0,    0],  dtype = int)

time_points = np.linspace(0, int(1*6.5*10), int(1*6.5*10))

iterations = 10

# See random number for reproducibility
np.random.seed(420)

# Initialise output array
out_variables = np.empty((iterations, len(time_points), len(initial_conditions)), dtype = int)

In [54]:
for index in tqdm(range(iterations)):
    out_variables[index, :, :] = gillespie_algorithim(SigB_propensity, SigB_acc_update, initial_conditions, time_points, args = args)
    
# out_variables = biocircuits.gillespie_ssa(SigB_propensity, SigB_acc_update, initial_conditions, time_points,
#                                          size = iterations, args = args, n_threads = 4)

  0%|          | 0/10 [00:00<?, ?it/s]

In [55]:
# Now we can plot the results
plots = [bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'P'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V_p'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'PV_p')]
         
for index in range(len(plots)):
    for out_variables_current in out_variables[:,:,index]:
        plots[index].line(time_points, out_variables_current, line_width = 0.3, alpha = 1, line_join = 'bevel')
    plots[index].line(time_points, out_variables[:,:,index].mean(axis = 0), line_width = 6, color = 'orange', line_join = 'bevel')
#Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols = 4))

In [56]:
#            v_0,  F,      K, lambda_W, lambda_V,    k_Bw,   k_Dw,    k_B1,    k_B2,    k_B3, k_B4,    k_B5,   k_D1,   k_D2,   k_D3,   k_D4,   k_D5,    k_K1,    k_K2,    k_P,  k_deg
# args = ( 1.44e-3, 30, 0.2e-6,        4,      4.5, 1.296e1, 6.48e4, 1.296e1, 1.296e1, 1.296e1, 6.48, 1.296e1, 6.48e4, 6.48e4, 6.48e4,   6.48, 6.48e4, 1.296e5, 1.296e5, 6.48e5, 2.52e3)
# args = ( 1.44e-3, 30, 0.2e-6,        4,      4.5, 1.296e7, 6.48e4, 1.296e7, 1.296e7, 1.296e7, 6.48, 1.296e1, 6.48e4, 6.48e4, 6.48e4, 6.48e6, 6.48e4, 1.296e5, 1.296e5, 6.48e5, 2.52e3)
#From Torkel v_0,  F,   K, lambda_W, lambda_V, k_Bw, k_Dw, k_B1, k_B2, k_B3, k_B4, k_B5, k_D1, k_D2,  k_D3, k_D4, k_D5, k_K1, k_K2, k_P,  k_deg
args = (     0.4, 30, 0.2,        4,      4.5, 3600,   18, 3600, 3600, 3600, 1800, 3600,   18,   18,    18, 1800,   18,   36,   36, 180, 0.7)
# Low Stress                    sig_B, W, V,  P, W_2, W_2V, W_2V_2, W_2sig_B, V_p, PV_p
initial_conditions = np.array([     0, 0, 0,  1,   0,    0,      0,        0,   0,    0],  dtype = int)
# High Stress                    sig_B, W, V,        P, W_2, W_2V, W_2V_2, W_2sig_B, V_p, PV_p
#initial_conditions = np.array([     0, 0, 0, 0.80e-6,   0,    0,      0,        0,   0,    0],  dtype = int)

time_points = np.linspace(0, int(1*6.5*10), int(1*6.5*10))

iterations = 10

# See random number for reproducibility
np.random.seed(420)

# Initialise output array
out_variables = np.empty((iterations, len(time_points), len(initial_conditions)), dtype = int)

In [57]:
for index in tqdm(range(iterations)):
    out_variables[index, :, :] = gillespie_algorithim(SigB_propensity, SigB_acc_update, initial_conditions, time_points, args = args)
    
# out_variables = biocircuits.gillespie_ssa(SigB_propensity, SigB_acc_update, initial_conditions, time_points,
#                                          size = iterations, args = args, n_threads = 4)

  0%|          | 0/10 [00:00<?, ?it/s]

In [58]:
# Now we can plot the results
plots = [bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'P'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V_p'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'PV_p')]
         
for index in range(len(plots)):
    for out_variables_current in out_variables[:,:,index]:
        plots[index].line(time_points, out_variables_current, line_width = 0.3, alpha = 1, line_join = 'bevel')
    plots[index].line(time_points, out_variables[:,:,index].mean(axis = 0), line_width = 6, color = 'orange', line_join = 'bevel')
#Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols = 4))

In [23]:
# Now we can plot the results
plots = [bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'P'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2V_2'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'W_2SigB'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'V_p'),
         bokeh.plotting.figure(plot_width = 300, plot_height = 200, x_axis_label = 'time', y_axis_label = 'PV_p')]
         
for index in range(len(plots)):
    for out_variables_current in out_variables[:,:,index]:
        plots[index].line(time_points, out_variables_current, line_width = 0.3, alpha = 1, line_join = 'bevel')
    plots[index].line(time_points, out_variables[:,:,index].mean(axis = 0), line_width = 6, color = 'orange', line_join = 'bevel')
#Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols = 4))